# Module Tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from tqdm.notebook import trange
import matplotlib.pyplot as plt

from q_agents import QAgent
from networks import QNet, ELMNet
from environment import Environment
from run_agent import do_run

ImportError: attempted relative import with no known parent package

In [4]:
def do_run(agent, env, N_ep):
    R_ep = []
    t = trange(N_ep, desc='bar_desc', leave=True)
    for ep_no in t:
        s = env.reset()
        done = False
        Rt = 0
        n_step = 0
        while not done:
            a = agent.action_select(s)
            s, r, done, _ = env.step(a)
            agent.update(s,r,done)
            Rt += r
            n_step +=1
        R_ep.append(Rt)
        t.set_description('Last reward: {}'.format(Rt))
        t.refresh()
    return R_ep, agent, env

In [6]:
N_ep = 800
env = Environment()
agent = QAgent(env, net_type='QNet', alpha = 0.0065, update_steps=70, N_hid=29, eps0=0.67, gamma=0.99, clip_norm=1.0, memory_size=10000, minibatch_size=26)
# agent = QAgent(env, net_type='ELMNet', update_steps=50, N_hid=28, eps0=0.66, gamma=0.66, memory_size=10000, minibatch_size=4)

R_ep, agent, env = do_run(agent, env, N_ep)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


KeyboardInterrupt: 

In [6]:
agent.nn.sess.close()

In [75]:
X = agent.memory.sample(50)

In [72]:
%%time
s = np.stack([d[0] for d in X])
a = np.array([d[1] for d in X])
r = np.array([d[2] for d in X])
Sd = np.stack([d[3] for d in X])
St = np.invert(np.array([d[4] for d in X]))

CPU times: user 677 µs, sys: 0 ns, total: 677 µs
Wall time: 436 µs


In [73]:
%%time
s, a, r, Sd, St = (np.stack([d[0] for d in X]),
              np.array([d[1] for d in X]),
              np.array([d[2] for d in X]),
              np.stack([d[3] for d in X]),
                   np.invert(np.array([d[4] for d in X])))

CPU times: user 315 µs, sys: 20 µs, total: 335 µs
Wall time: 259 µs


In [5]:
net1 = ELMNet(4,2)
net2 = ELMNet(4,2)

In [8]:
params1 = net1.get_params()
params1

{'W': array([[0.5513873 , 0.99018204],
        [0.26243877, 0.6029563 ]], dtype=float32),
 'w_in': array([[0.01396406, 0.920694  ],
        [0.8457651 , 0.9497851 ],
        [0.65000665, 0.6344228 ],
        [0.8699683 , 0.43895304]], dtype=float32),
 'b_in': array([[0., 0.]], dtype=float32)}

In [9]:
params2 = net2.get_params()
params2

{'W': array([[0.12305117, 0.5535748 ],
        [0.8529272 , 0.39881754]], dtype=float32),
 'w_in': array([[0.6971314 , 0.6187943 ],
        [0.408126  , 0.61991894],
        [0.6689235 , 0.99879   ],
        [0.28883588, 0.44413984]], dtype=float32),
 'b_in': array([[0., 0.]], dtype=float32)}

In [11]:
net2.assign_params(params1)
net2.get_params()

{'W': array([[0.5513873 , 0.99018204],
        [0.26243877, 0.6029563 ]], dtype=float32),
 'w_in': array([[0.01396406, 0.920694  ],
        [0.8457651 , 0.9497851 ],
        [0.65000665, 0.6344228 ],
        [0.8699683 , 0.43895304]], dtype=float32),
 'b_in': array([[0., 0.]], dtype=float32)}